# Section 5.1 - Reproduction using SGLD

# Mixture of Gaussians with Stochastic Gradient Langevin Dynamics (SGLD)
This notebook reproduces the example in the SGLD paper using a bimodal Gaussian mixture.


In [ ]:
#1. Setup and Data Generation

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Ground truth parameters
theta1_true = 0
theta2_true = 1
sigma_x = np.sqrt(2)

# Generate data
np.random.seed(42)
N = 100
z = np.random.binomial(1, 0.5, N)
x = z * np.random.normal(theta1_true + theta2_true, sigma_x, N) + (1 - z) * np.random.normal(theta1_true, sigma_x, N)

# Prior std
sigma1 = np.sqrt(10)
sigma2 = 1

In [ ]:
#2. SGLD Sampling
steps = 10000
epsilon_schedule = lambda t: 0.01 * (1 + t)**-0.55
theta = np.array([0.0, 0.0])
samples = []

for t in tqdm(range(steps)):
    i = np.random.randint(0, N)
    xi = x[i]
    mu1 = theta[0]
    mu2 = theta[0] + theta[1]
    p1 = np.exp(-(xi - mu1)**2 / (2 * sigma_x**2))
    p2 = np.exp(-(xi - mu2)**2 / (2 * sigma_x**2))
    p1, p2 = p1 / (p1 + p2), p2 / (p1 + p2)
    grad_log_lik = np.array([
        (p1 * (xi - mu1) + p2 * (xi - mu2)) / (sigma_x**2),
        p2 * (xi - mu2) / (sigma_x**2)
    ])
    grad_log_prior = np.array([-theta[0] / sigma1**2, -theta[1] / sigma2**2])
    grad = grad_log_lik + grad_log_prior
    eps = epsilon_schedule(t)
    theta += 0.5 * eps * grad + np.sqrt(eps) * np.random.randn(2)
    samples.append(theta.copy())

samples = np.array(samples)

In [ ]:
#3. Posterior Visualization
g = sns.jointplot(x=samples[:, 0], y=samples[:, 1], kind="kde")
g.set_axis_labels("$\\theta_1$", "$\\theta_2$")
g.fig.suptitle("Joint Posterior (KDE)", y=1.02)
plt.show()

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
sns.histplot(samples[:, 0], bins=50, kde=True)
plt.title("Histogram of $\theta_1$")
plt.xlabel("$\\theta_1$")

plt.subplot(1, 2, 2)
sns.histplot(samples[:, 1], bins=50, kde=True)
plt.title("Histogram of $\theta_2$")
plt.xlabel("$\\theta_2$")
plt.tight_layout()
plt.show()

In [ ]:
#4. Trace Plots
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(samples[:, 0])
plt.title("Trace plot for $\theta_1$")
plt.subplot(1, 2, 2)
plt.plot(samples[:, 1])
plt.title("Trace plot for $\theta_2$")
plt.tight_layout()
plt.show()

In [ ]:
#5. Posterior Summary Statistics
mean_theta = samples.mean(axis=0)
cov_theta = np.cov(samples.T)
corr = np.corrcoef(samples.T)

print(f"Mean θ: {mean_theta}")
print(f"Covariance matrix:\n{cov_theta}")
print(f"Correlation matrix:\n{corr}")

In [ ]:
#6. Diagnostics: Autocorrelation, ESS, R̂
from statsmodels.tsa.stattools import acf
import pandas as pd

def plot_autocorrelation(samples, lags=100):
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    for i, theta_label in enumerate(["θ₁", "θ₂"]):
        autocorr = acf(samples[:, i], nlags=lags)
        axes[i].stem(range(lags + 1), autocorr)
        axes[i].set_title(f"Autocorrelation of {theta_label}")
        axes[i].set_xlabel("Lag")
        axes[i].set_ylabel("ACF")
    plt.tight_layout()
    plt.show()

def compute_ess(samples):
    ess = []
    for i in range(samples.shape[1]):
        acf_vals = acf(samples[:, i], nlags=100, fft=True)
        ess_i = len(samples) / (1 + 2 * np.sum(acf_vals[1:]))
        ess.append(ess_i)
    return ess

def compute_rhat(samples):
    split_chains = np.array_split(samples, 2, axis=0)
    chains = [split_chains[0], split_chains[1]]
    m = len(chains)
    n = len(chains[0])
    means = np.array([np.mean(chain, axis=0) for chain in chains])
    variances = np.array([np.var(chain, axis=0, ddof=1) for chain in chains])
    B = n * np.var(means, axis=0, ddof=1)
    W = np.mean(variances, axis=0)
    var_hat = ((n - 1) / n) * W + B / n
    R_hat = np.sqrt(var_hat / W)
    return R_hat

plot_autocorrelation(samples)
ess_values = compute_ess(samples)
rhat_values = compute_rhat(samples)

pd.DataFrame({
    "Parameter": [r"$\theta_1$", r"$\theta_2$"],
    "ESS": ess_values,
    "R_hat": rhat_values
})

In [ ]:
#7. Log Joint Probability Plot
# Log joint probability over time
def log_joint_prob(theta, x, sigma1, sigma2, sigma_x):
    log_prior = -0.5 * (theta[0]**2 / sigma1**2 + theta[1]**2 / sigma2**2)
    mu1 = theta[0]
    mu2 = theta[0] + theta[1]
    log_lik = 0
    for xi in x:
        p1 = np.exp(-(xi - mu1)**2 / (2 * sigma_x**2))
        p2 = np.exp(-(xi - mu2)**2 / (2 * sigma_x**2))
        log_lik += np.log(0.5 * p1 + 0.5 * p2)
    return log_prior + log_lik

log_probs = [log_joint_prob(theta, x, sigma1, sigma2, sigma_x) for theta in samples]

plt.plot(log_probs)
plt.xlabel("Iteration")
plt.ylabel("Log Joint Probability")
plt.title("Log Joint Probability over Iterations")
plt.grid(True)
plt.show()

In [ ]:
#8. Multiple Chains KDE
# Run multiple chains
num_chains = 5
chain_samples = []

for seed in range(num_chains):
    np.random.seed(seed)
    theta = np.random.randn(2)  # different init
    chain = []
    for t in range(steps):
        i = np.random.randint(0, N)
        xi = x[i]
        mu1 = theta[0]
        mu2 = theta[0] + theta[1]
        p1 = np.exp(-(xi - mu1)**2 / (2 * sigma_x**2))
        p2 = np.exp(-(xi - mu2)**2 / (2 * sigma_x**2))
        p1, p2 = p1 / (p1 + p2), p2 / (p1 + p2)
        grad_log_lik = np.array([
            (p1 * (xi - mu1) + p2 * (xi - mu2)) / (sigma_x**2),
            p2 * (xi - mu2) / (sigma_x**2)
        ])
        grad_log_prior = np.array([-theta[0] / sigma1**2, -theta[1] / sigma2**2])
        grad = grad_log_lik + grad_log_prior
        eps = epsilon_schedule(t)
        theta += 0.5 * eps * grad + np.sqrt(eps) * np.random.randn(2)
        chain.append(theta.copy())
    chain_samples.append(np.array(chain))

# Overlay KDEs of all chains
plt.figure(figsize=(6, 6))
for i, chain in enumerate(chain_samples):
    sns.kdeplot(x=chain[:, 0], y=chain[:, 1], label=f"Chain {i+1}", alpha=0.6)
plt.title("Posterior KDEs from Multiple Chains")
plt.legend()
plt.show()

In [ ]:
#9. Rejection Rate Scaling
# Rejection rate simulation for fixed steps
from scipy.spatial.distance import euclidean

fixed_steps = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
divergences = []

for fixed_eps in fixed_steps:
    theta = np.zeros(2)
    trajectory = []
    for t in range(1000):
        i = np.random.randint(0, N)
        xi = x[i]
        mu1 = theta[0]
        mu2 = theta[0] + theta[1]
        p1 = np.exp(-(xi - mu1)**2 / (2 * sigma_x**2))
        p2 = np.exp(-(xi - mu2)**2 / (2 * sigma_x**2))
        p1, p2 = p1 / (p1 + p2), p2 / (p1 + p2)
        grad_log_lik = np.array([
            (p1 * (xi - mu1) + p2 * (xi - mu2)) / (sigma_x**2),
            p2 * (xi - mu2) / (sigma_x**2)
        ])
        grad_log_prior = np.array([-theta[0] / sigma1**2, -theta[1] / sigma2**2])
        grad = grad_log_lik + grad_log_prior
        noise = np.random.randn(2)
        proposal = theta + 0.5 * fixed_eps * grad + np.sqrt(fixed_eps) * noise
        dist = euclidean(theta, proposal)
        trajectory.append(dist)
        theta = proposal
    divergences.append(np.mean(trajectory))

plt.plot(np.log10(fixed_steps), divergences, marker='o')
plt.xlabel("log10(Step Size)")
plt.ylabel("Avg. Step Distance (proxy for rejection)")
plt.title("Rejection Rate Proxy vs Step Size")
plt.grid(True)
plt.show()